# ❄️ Snowflake Arctic Quickstart with Cortex LLM Functions

In this quickstart you will learn build and evaluate a RAG application with Snowflake Arctic.

Building and evaluating RAG applications with Snowflake Arctic offers developers a unique opportunity to leverage a top-tier, enterprise-focused LLM that is both cost-effective and open-source. Arctic excels in enterprise tasks like SQL generation and coding, providing a robust foundation for developing intelligent applications with significant cost savings. [Learn more about Snowflake Arctic](https://www.snowflake.com/blog/arctic-open-efficient-foundation-language-models-snowflake/)

In this example, we will use Arctic Embed (`snowflake-arctic-embed-m`) as our embedding model via HuggingFace, and Arctic, a 480B hybrid MoE LLM for both generation and as the LLM to power TruLens feedback functions. The Arctic LLM is fully-mananaged by [Cortex LLM functions](https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions)

Note, you'll need to have an [active Snowflake account](https://signup.snowflake.com/
) to run Cortex LLM functions from Snowflake's data warehouse.
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/models/arctic_quickstart.ipynb)

In [ ]:
# ! pip install trulens_eval chromadb sentence-transformers snowflake-snowpark-python

In [1]:
import os
os.environ["SNOWFLAKE_ACCOUNT"] = "xxx-xxx" # xxx-xxx.snowflakecomputing.com
os.environ["SNOWFLAKE_USER"] = "..." 
os.environ["SNOWFLAKE_USER_PASSWORD"] = "..."

In [4]:
from trulens.utils.keys import check_keys
check_keys("SNOWFLAKE_ACCOUNT", "SNOWFLAKE_USER", "SNOWFLAKE_USER_PASSWORD")
from snowflake.snowpark import Session

connection_params = {
    "account": os.environ["SNOWFLAKE_ACCOUNT"],
    "user": os.environ["SNOWFLAKE_USER"],
    "password": os.environ["SNOWFLAKE_USER_PASSWORD"],
}


# Create a Snowflake session
snowflake_session = Session.builder.configs(connection_params).create()

Package json not present in requirements.
No .env found in /Users/dhuang/Documents/git/trulens/trulens_eval/examples/expositional/models or its parents. You may need to specify secret keys in another manner.


✅ Key SNOWFLAKE_ACCOUNT set from environment.
✅ Key SNOWFLAKE_USER set from environment.
✅ Key SNOWFLAKE_USER_PASSWORD set from environment.


## Get Data

In this case, we'll just initialize some simple text in the notebook.

In [5]:
university_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

## Create Vector Store

Create a chromadb vector store in memory.

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("Snowflake/snowflake-arctic-embed-m")

In [6]:
document_embeddings = model.encode([university_info])

In [8]:
import chromadb

chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(name="Universities")

Add the university_info to the embedding database.

In [9]:
vector_store.add("uni_info",
                 documents=university_info,
                 embeddings=document_embeddings)

## Build RAG from scratch

Build a custom RAG from scratch, and add TruLens custom instrumentation.

In [10]:
from trulens.core import Tru
from trulens.core.tru_custom_app import instrument
tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [11]:
import json

class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(
            query_embeddings=model.encode(
            [query], prompt_name="query"),
            n_results=2
        )
        return results['documents']

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        
        def escape_string_for_sql(input_string):
            escaped_string = input_string.replace('\\', '\\\\')
            escaped_string = escaped_string.replace("'", "''")
            return escaped_string
        
        prompt=escape_string_for_sql(f"""
         We have provided context information below. 
            {context_str}
            Given this information, please answer the question: {query}
        """)
 
        res = snowflake_session.sql(f"""SELECT SNOWFLAKE.CORTEX.COMPLETE(
            'snowflake-arctic',
            [
            {{'role': 'user', 'content': '{prompt}'}}
            ], {{
                'temperature': 0
            }}
            )""").collect()
       
        completion = json.loads(res[0][0])["choices"][0]["messages"]
        print("full response from cortex function:")
        print(res)
        return completion

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion

rag = RAG_from_scratch()

### Dev Note as of June 2024: 
Alternatively, we can use Cortex's Python API ([documentation](https://docs.snowflake.com/user-guide/snowflake-cortex/llm-functions#using-snowflake-cortex-llm-functions-with-python)) directly to have cleaner interface and avoid constructing SQL commands ourselves. 
The reason we are invoking the SQL function directly via `snowflake_session.sql()` is that the response from Cortex's Python API is still experimental and not as feature-rich as the one from SQL function as of the time of writing. i.e. inconsistency issues with structured json outputs and missing usage information have been observed, lack of support for advanced chat-style (multi-message), etc.
Below is a minimal example of using Python API instead. 





In [10]:

# from snowflake.cortex import Complete
# def complete(user_query) -> str:
#     completion = Complete(
#         model="snowflake-arctic",
#         prompt=f"[FILL IN SYSTEM PROMPTS IF NEEDED ]{user_query}",
#         session=snowflake_session,
#     )
#     return completion


## Set up feedback functions.

Here we'll use groundedness, answer relevance and context relevance to detect hallucination.

In [12]:
from trulens.core import Feedback, Select
from trulens.external.provider import Cortex

import numpy as np

# Initialize LiteLLM-based feedback function collection class:
provider = Cortex(model_engine="snowflake-arctic")

# Define a groundedness feedback function
f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

f_coherence = (
    Feedback(provider.coherence_with_cot_reasons, name = "coherence")
    .on_output()
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input context will be set to __record__.app.retrieve.rets.collect() .
✅ In coherence, input text will be set to __record__.main_output or `Select.RecordOutput` .


## Construct the app
Wrap the custom RAG with TruCustomApp, add list of feedbacks for eval

In [13]:
from trulens.core import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG v1',
    feedbacks = [f_groundedness, f_answer_relevance, f_context_relevance, f_coherence])

## Run the app
Use `tru_rag` as a context manager for the custom RAG-from-scratch app.

In [14]:
with tru_rag as recording:
    resp = rag.query("When is University of Washington founded?")

full response from cortex function:
[Row(SNOWFLAKE.CORTEX.COMPLETE(
            'SNOWFLAKE-ARCTIC',
            [
            {'ROLE': 'USER', 'CONTENT': '
         WE HAVE PROVIDED CONTEXT INFORMATION BELOW. 
            [[''\\NTHE UNIVERSITY OF WASHINGTON, FOUNDED IN 1861 IN SEATTLE, IS A PUBL='{\n  "choices": [\n    {\n      "messages": " The University of Washington was founded in 1861."\n    }\n  ],\n  "created": 1718315813,\n  "model": "snowflake-arctic",\n  "usage": {\n    "completion_tokens": 14,\n    "prompt_tokens": 148,\n    "total_tokens": 162\n  }\n}')]


In [15]:
resp

' The University of Washington was founded in 1861.'

In [23]:
tru.get_leaderboard(app_ids=["RAG v1"])

,Groundedness,Context Relevance,coherence,Answer Relevance,latency,total_cost
app_id,,,,,,
RAG v1,1.0,1.0,1.0,1.0,2.0,0.0


In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard(tru)